<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [21]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [22]:
# util.load_hopskey()

In [23]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "m5uQYs9YDv3Il4N2.qSL9mUQnbvqxls4lZV6GTYaFEaTBAjHwWnQZoLBGuq1HpE3m4DVD6EsLq671AVxP"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160345
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "Germany", "city": "Eimsb\\u00fcttel", "street": "Rentzelstra\\u00dfe", "aqicn_url": "https://api.waqi.info/feed/A83758", "latitude": 53.5649263, "longitude": 9.9777639}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [24]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=3,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=3,
)

In [25]:
# Retrieve feature groups
air_quality_fg_1 = fs.get_feature_group(
    name='air_quality',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [26]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df_1 = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)

In [27]:
aq_df = air_quality_fg.read()
aq_df.sort_values("date", inplace=True)
aq_today_df["pm25_lagging"] = aq_df.tail(3)["pm25"].mean()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.17s) 


In [28]:
aq_today_df_1

,pm25,country,city,street,date,url
0,44.0,Germany,Eimsbüttel,Rentzelstraße,2024-11-12,https://api.waqi.info/feed/A83758


In [29]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pm25          1 non-null      float32       
 1   country       1 non-null      object        
 2   city          1 non-null      object        
 3   street        1 non-null      object        
 4   date          1 non-null      datetime64[ns]
 5   url           1 non-null      object        
 6   pm25_lagging  1 non-null      float32       
dtypes: datetime64[ns](1), float32(2), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [30]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 53.5°N 10.0°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-12,9.05,0.0,9.220499,38.659828,Eimsbüttel
1,2024-11-13,7.75,0.1,9.885262,236.888641,Eimsbüttel
2,2024-11-14,10.55,0.0,21.437386,310.914276,Eimsbüttel
3,2024-11-15,10.80,0.0,14.404499,268.567932,Eimsbüttel
4,2024-11-16,9.65,0.0,21.971800,251.861816,Eimsbüttel
5,2024-11-17,8.00,0.0,15.484185,197.592514,Eimsbüttel
6,2024-11-18,13.35,0.2,24.162945,183.416519,Eimsbüttel
7,2024-11-19,8.10,0.1,34.200001,233.130020,Eimsbüttel
8,2024-11-20,4.80,0.0,19.008547,155.376450,Eimsbüttel
9,2024-11-21,9.40,0.0,35.652725,223.363464,Eimsbüttel


In [31]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [32]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-12 08:50:14,785 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160345/fs/1151048/fg/1341778


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160345/jobs/named/air_quality_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3002a1ff0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 665607
         }
       },
       "result": {
         "observed_value": 44.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T07:50:14.000785Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [33]:
# Insert new data
air_quality_fg_1.insert(aq_today_df_1)

2024-11-12 08:50:26,011 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160345/fs/1151048/fg/1340791


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160345/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3002ede10>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 664605
         }
       },
       "result": {
         "observed_value": 44.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T07:50:26.000010Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [34]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-12 08:50:36,284 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160345/fs/1151048/fg/1342795


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160345/jobs/named/weather_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3002a1a80>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 666626
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T07:50:36.000283Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
